## Imports


In [7]:
# If needed, mount Google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
!pip install mne
import mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 14.3 MB/s 


## Define Electrodes

In [12]:
def get_electrodes_as_list(src_txt_file):
  file =  src_txt_file
  infile = open(file, 'r')
  lines = infile.readlines() 
  list_of_electrodes = []

  for line in lines: #lines is a list with each item representing a line of the file
    #if 'control' in line:
    line = line.split('\\')[0]
    list_of_electrodes.append(line.split('\n')[0].replace("'","")) #print lines for control condition

  
  infile.close()

  return list_of_electrodes

In [13]:
#insert the location of a txt containing the list of electrodes as rows. this is the list of electrodes before any processing
electrode_order = get_electrodes_as_list('/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/channel_locs.txt')
if electrode_order[-1] == '[]':
  electrode_order.pop(-1)
print(electrode_order)

['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8', 'VEOG', 'X', 'Y', 'Z}']


In [14]:
#specify the target electrode order
#target_electrode_order  = ['Fp1','Fz','F3','F7','FT9','FC5','FC1','C3','T7','TP9','CP5','CP1','Pz','P3','P7','O1','Oz','O2','P4','P8','TP10','CP6','CP2','Cz','C4','T8','FT10','FC6','FC2','F4','F8','Fp2','AF7','AF3','AFz','F1','F5','FT7','FC3','FCz','C1','C5','TP7','CP3','P1','P5','PO7','PO3','POz','PO4','PO8','P6','P2','CP4','TP8','C6','C2','FC4','FT8','F6','F2','AF4','AF8']
target_electrode_order = [ 'Fp1' ,'Fz' ,'F3', 'F7' ,'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3' ,'P7', 'O1', 'Oz', 'O2' ,'P4', 'P8', 'TP10' ,'CP6' ,'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7','FC3', 'FCz', 'C1' ,'C5', 'TP7', 'CP3' ,'P1', 'P5', 'PO7', 'POz', 'PO8', 'P6', 'P2', 'CP4', 'TP8' ,'C6' ,'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4' ,'AF8']

In [18]:
#if you want to change references, here is where you do it, otherwise keep it the same.
src_reference_channel = 'CPz'
target_reference_channel = 'CPz'

for electrode in electrode_order:
  if electrode == 'CPz':
    assert False, 'Houston, we have a problem'
  else:
    if electrode == electrode_order[-1]:
      print('good to go')

good to go


In [20]:
#specify the directory containing all the csv files you want
src_dir = '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/'
src_files = glob.glob(src_dir + "*.csv")
print(src_files)#src_files.reverse()

['/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_801.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_802.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_804.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_805.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_806.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_807.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_808.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_809.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD/PD_810.csv', '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/exte

In [21]:
#specify the directory where you want to put the rereferenced data
target_dir = '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/external_validation/New_Mexico_ON_MEDS/PD_reref/'

In [ ]:
# for trouble shooting you can choose how many rows to see
# pd.set_option('display.max_rows', 10)

## loop through each file, rereferencing and processing

In [25]:
#read in the file 
for file in src_files:
  
  #read in the data as a df, then specify whichever row or column contains the time measurements
  df = pd.read_csv(file, sep=',',header=None)
  time_measurements = df.iloc[0] #df.columns #
  
  #manually specify any rows that are not EEG electrodes, including time, gyroscope etc. 
  to_drop = [0, 64, 65, 66, 67]
  df = df.drop(to_drop, axis=0)
  
  #intentionally left blank, will construct a subset list of only electrodes, no time, gyroscope etc. 
  labels = []
  for i in range(len(electrode_order)- (len(to_drop)-1)): #subtract the number of fields you had to remove (len(electrode_order) - len(target_electrode_order))
    labels.append(electrode_order[i])
  
  #assign that list of labels to the index of your electrodes
  df.index = labels
  
  #create mne info object
  sampling_period = (time_measurements[1]-time_measurements[0])/1000 #assuming time in ms
  mne_info = mne.create_info(labels[0:len(df)], sfreq=(1/sampling_period), ch_types='eeg')
  
  #use info object to convert the df (which at this point is only electrode data), add the reference electrode since it is usually omitted, then specify the montage
  data_mne = mne.io.RawArray(df, mne_info)
  data_mne.add_reference_channels(src_reference_channel)
  data_mne.set_montage('standard_1020')
  
  #now that you have an empty reference channel row, you can re-reference the data and drop the new reference
  rereferenced_data, ref_data = mne.set_eeg_reference(data_mne, ref_channels=[target_reference_channel])
  rereferenced_df = rereferenced_data.to_data_frame(scalings=dict(eeg=1))
  rereferenced_df.drop(columns=target_reference_channel)

  #intentionally left blank. this loop will find which of the target electrodes are not present in your re-referenced data
  to_remove = []
  desired_electrode_order = target_electrode_order
  for i in range(len(desired_electrode_order)):
    if desired_electrode_order[i] not in rereferenced_df.columns:
      print('removing')
      to_remove.append(i)

  #you need to reverse this list so you can pop them off the list of electrodes without causing reading frame shift
  to_remove.reverse()
  for i in range(len(to_remove)):
    desired_electrode_order.pop(to_remove[i])

  #subsample your electrodes to keep only the ones you want
  rereferenced_df = rereferenced_df[desired_electrode_order]

  #convert time measurements to pd.series then insert into your df
  time_pd_data = {'time':time_measurements}
  time_pd_series = pd.Series(time_pd_data)
  rereferenced_df.insert(0,'time',time_measurements)
  #display(rereferenced_df)

  #construct a filename by adding the directory with the filename
  target_filename = target_dir + file.split('/')[-1]
  #print(target_filename)

  rereferenced_df.to_csv(path_or_buf=target_filename, sep=',') #note that this will 1) insert a row containing the index as column zero 2) overwrite a file with the same name

  
  
 

Creating RawArray with float64 data, n_channels=63, n_times=93501
    Range : 0 ... 93500 =      0.000 ...   187.000 secs
Ready.
Location for this channel is unknown; consider calling set_montage() again if needed.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=63, n_times=91001
    Range : 0 ... 91000 =      0.000 ...   182.000 secs
Ready.
Location for this channel is unknown; consider calling set_montage() again if needed.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=63, n_times=96501
    Range : 0 ... 96500 =      0.000 ...   193.000 secs
Ready.
Location for this channel is unknown; consider calling set_montage() again if needed.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Creating RawArray with float64 data, n_channels=63, n_times=105001
    Range : 0 ... 105000 =      0.000 .

In [26]:
print(rereferenced_df.columns.values)

['time' 'Fp1' 'Fz' 'F3' 'F7' 'FC5' 'FC1' 'C3' 'T7' 'TP9' 'CP5' 'CP1' 'Pz'
 'P3' 'P7' 'O1' 'Oz' 'O2' 'P4' 'P8' 'TP10' 'CP6' 'CP2' 'Cz' 'C4' 'T8'
 'FT10' 'FC6' 'FC2' 'F4' 'F8' 'Fp2' 'AF7' 'AF3' 'AFz' 'F1' 'F5' 'FT7'
 'FC3' 'FCz' 'C1' 'C5' 'TP7' 'CP3' 'P1' 'P5' 'PO7' 'POz' 'PO8' 'P6' 'P2'
 'CP4' 'TP8' 'C6' 'C2' 'FC4' 'FT8' 'F6' 'F2' 'AF4' 'AF8']


## compare to original dataset

In [ ]:
original_data_src = '/content/drive/MyDrive/UTOR-MSc/data/EEG_PD_data/data_csv/ds003490-000015.csv'

In [ ]:
original_df = pd.read_csv(original_data_src,sep=',')
display(original_df)

Output hidden; open in https://colab.research.google.com to view.

## Visualize

In [ ]:
original_df = pd.read_csv(src_files[0],sep=',')
display(original_df)

,0,2,4,6,8,10,12,14,16,18,...,1.9198e+05.3,1.9198e+05.4,1.9199e+05,1.9199e+05.1,1.9199e+05.2,1.9199e+05.3,1.9199e+05.4,1.92e+05,1.92e+05.1,1.92e+05.2
0,-11514.0,-11510.0,-11511.0,-11514.0,-11513.0,-11510.0,-11509.0,-11508.0,-11509.0,-11510.0,...,-12372.0,-12377.0,-12384.0,-12387.0,-12388.0,-12388.0,-12386.0,-12379.0,-12382.0,-12387.0
1,6353.7,6353.5,6354.6,6352.9,6352.3,6352.7,6352.9,6351.8,6353.2,6353.6,...,7138.9,7137.4,7137.3,7134.2,7133.2,7133.3,7134.4,7137.7,7142.1,7141.7
2,10780.0,10792.0,10791.0,10787.0,10786.0,10786.0,10787.0,10785.0,10785.0,10785.0,...,10590.0,10588.0,10587.0,10584.0,10583.0,10584.0,10585.0,10593.0,10614.0,10594.0
3,-10135.0,-10134.0,-10134.0,-10135.0,-10134.0,-10132.0,-10132.0,-10133.0,-10134.0,-10135.0,...,-11800.0,-11806.0,-11809.0,-11810.0,-11812.0,-11813.0,-11813.0,-11811.0,-11813.0,-11812.0
4,-9250.4,-9250.8,-9251.0,-9251.2,-9253.0,-9250.6,-9248.5,-9250.9,-9258.1,-9255.7,...,-9761.4,-9760.0,-9760.0,-9767.9,-9772.2,-9776.6,-9779.4,-9779.8,-9780.2,-9776.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,-3763.6,-3764.6,-3764.3,-3764.2,-3765.0,-3763.2,-3761.8,-3760.0,-3755.8,-3754.3,...,-3500.3,-3503.5,-3505.0,-3508.2,-3509.0,-3508.0,-3509.9,-3508.0,-3506.4,-3506.2
63,-1300500.0,-1300200.0,-1301400.0,-1303600.0,-1306300.0,-1307700.0,-1306500.0,-1304100.0,-1301400.0,-1299400.0,...,-1347600.0,-1353500.0,-1364200.0,-1373700.0,-1377300.0,-1373000.0,-1362600.0,-1352000.0,-1347400.0,-1351700.0
64,-993950.0,-994620.0,-994950.0,-994990.0,-994570.0,-993510.0,-991980.0,-989890.0,-988950.0,-990150.0,...,-825200.0,-825370.0,-825960.0,-826400.0,-826900.0,-827340.0,-826940.0,-826480.0,-825980.0,-826020.0
65,-489350.0,-489730.0,-490240.0,-491060.0,-492260.0,-493200.0,-493450.0,-493140.0,-492590.0,-492000.0,...,-29588.0,-29551.0,-29510.0,-29203.0,-28593.0,-27472.0,-26737.0,-26575.0,-26317.0,-26183.0


In [ ]:
#visualize the end product
display(rereferenced_df)

,time,Fp1,Fz,F3,F7,FC5,FC1,C3,T7,TP9,...,CP4,TP8,C6,C2,FC4,FT8,F6,F2,AF4,AF8
0,0.0,-150.400,-39.773,-72.460,-59.6990,8.667,-24.627,-10.7870,-35.6740,27.69660,...,12.595,10.805,5.6910,-17.056,-25.995,-98.320,-107.600,-28.021,-67.640,-104.9200
1,2.0,-151.952,-32.169,-69.242,-58.7320,16.642,-16.736,0.6200,-20.3740,37.62810,...,17.845,16.242,11.9550,-11.461,-21.130,-95.682,-105.552,-25.448,-66.272,-136.1420
2,4.0,-162.285,-31.842,-67.635,-57.9650,19.293,-13.729,3.3820,-16.1600,47.36120,...,20.311,21.287,17.5340,-10.995,-19.000,-92.955,-104.045,-27.025,-73.105,-138.7450
3,6.0,-174.906,-35.127,-68.766,-61.2560,18.380,-15.556,0.6770,-17.7080,51.08700,...,20.481,22.473,17.3020,-12.921,-20.971,-91.006,-101.996,-28.943,-81.406,-132.4160
4,8.0,-183.676,-39.632,-70.526,-64.5360,15.611,-20.117,-2.2240,-19.3530,46.75598,...,17.674,20.096,13.4090,-16.213,-26.555,-94.236,-104.886,-32.228,-86.966,-139.1260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63255,126510.0,15.362,10.918,1.513,-9.8630,-11.112,-1.297,-16.3064,-20.4934,-48.71800,...,-9.364,-8.997,-30.5702,6.358,-9.167,4.092,3.973,11.637,-3.692,30.5610
63256,126510.0,18.204,12.144,-0.136,-14.8791,-11.395,-1.683,-14.7321,-21.7614,-47.82200,...,-8.281,-8.063,-29.1953,8.508,-7.645,7.376,7.353,11.838,1.206,21.2790
63257,126510.0,29.394,14.590,-0.028,-12.2300,-9.238,-0.313,-9.9370,-28.2048,-45.99900,...,-10.715,-9.522,-30.0184,7.190,-6.808,9.730,10.537,10.768,9.715,20.6890
63258,126520.0,41.996,21.185,3.644,-2.6000,1.218,8.625,5.4030,-28.1041,-37.04900,...,-8.704,-7.367,-28.2536,10.559,-1.920,9.936,10.596,13.654,14.511,-8.6070
